In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [10]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [11]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=False,
    c1=1e-4,
    tau=0.5,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.5545493364334106
epoch:  1, loss: 0.08372308313846588
epoch:  2, loss: 0.03995230048894882
epoch:  3, loss: 0.026942646130919456
epoch:  4, loss: 0.01772207021713257
epoch:  5, loss: 0.013669883832335472
epoch:  6, loss: 0.012841365300118923
epoch:  7, loss: 0.010877660475671291
epoch:  8, loss: 0.010600652545690536
epoch:  9, loss: 0.009986817836761475
epoch:  10, loss: 0.009718048386275768
epoch:  11, loss: 0.009625028818845749
epoch:  12, loss: 0.009317740797996521
epoch:  13, loss: 0.009213874116539955
epoch:  14, loss: 0.00904080644249916
epoch:  15, loss: 0.00897318497300148
epoch:  16, loss: 0.008603876456618309
epoch:  17, loss: 0.008455225266516209
epoch:  18, loss: 0.008254572749137878
epoch:  19, loss: 0.007989335805177689
epoch:  20, loss: 0.00769325764849782
epoch:  21, loss: 0.007504921872168779
epoch:  22, loss: 0.007323227822780609
epoch:  23, loss: 0.007184075657278299
epoch:  24, loss: 0.006882842164486647
epoch:  25, loss: 0.006721580866724253
epoc

In [12]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9734574527057158
Test metrics:  R2 = 0.9645880973329118


In [13]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].cpu().detach().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.06564666330814362
epoch:  1

/home/eugeniolr/Documents/invAI/torch_numopt/src/torch_numopt/utils.py:33: UserWarning: torch.linalg.svd: During SVD computation with the selected cusolver driver, batches 0 failed to converge. A more accurate method will be used to compute the SVD as a fallback. Check doc at https://pytorch.org/docs/stable/generated/torch.linalg.svd.html (Triggered internally at /pytorch/aten/src/ATen/native/cuda/linalg/BatchLinearAlgebraLib.cpp:690.)
  U, S, Vt = torch.linalg.svd(mat)


, loss: 0.03889923542737961
epoch:  2, loss: 0.03779246285557747
epoch:  3, loss: 0.020640326663851738
epoch:  4, loss: 0.014044285751879215
epoch:  5, loss: 0.009291606023907661
epoch:  6, loss: 0.008633079007267952
epoch:  7, loss: 0.008133187890052795
epoch:  8, loss: 0.007452551741153002
epoch:  9, loss: 0.00661282055079937
epoch:  10, loss: 0.005887879990041256
epoch:  11, loss: 0.005394181702286005
epoch:  12, loss: 0.00501747103407979
epoch:  13, loss: 0.004645932000130415
epoch:  14, loss: 0.004267852287739515
epoch:  15, loss: 0.003881315002217889
epoch:  16, loss: 0.0034876184072345495
epoch:  17, loss: 0.0030859021935611963
epoch:  18, loss: 0.002737503033131361
epoch:  19, loss: 0.0024413124192506075
epoch:  20, loss: 0.002185117918998003
epoch:  21, loss: 0.0019957709591835737
epoch:  22, loss: 0.0018404260044917464
epoch:  23, loss: 0.0013938195770606399
epoch:  24, loss: 0.0010860806796699762
epoch:  25, loss: 0.0009699227521196008
epoch:  26, loss: 0.0009068384533748031

In [14]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9935522849884326
Test metrics:  R2 = 0.9885562738980905
